In [1]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
from string import Template
import time
import random
import os
import re
import json

In [ ]:
good_reviews = Template("https://www.kinopoisk.ru/film/${film_id}/reviews/ord/date/status/good/perpage/200/page/${page_num}/")
bad_reviews = Template("https://www.kinopoisk.ru/film/${film_id}/reviews/ord/date/status/bad/perpage/200/page/${page_num}/")
url = "https://www.kinopoisk.ru/top/navigator/m_act[rating]/1%3A/order/num_vote/perpage/200/#results"

In [ ]:
def create_folders():
    path_bad = os.path.join("dataset", "bad")
    if not os.path.isdir(path_bad):
        os.makedirs(path_bad)
    path_good = os.path.join("dataset", "good")
    if not os.path.isdir(path_good):
        os.makedirs(path_good)

def slep():
    delay = random.uniform(10, 60)
    print(f'delay is: {delay}')
    time.sleep(delay)

In [ ]:
chrome = UserAgent().chrome

def get_page(link: str) -> str:
    global chrome
    slep()
    html_text = requests.get(link, headers={'User-Agent': chrome}).text
    return html_text

In [ ]:
create_folders()
films = {}
with open('films.json', 'r') as file:
    films = json.load(file)